## Installs

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [35]:
#PacMan dependancies
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
from random import sample 

## Imports

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64

from IPython.display import HTML
from IPython import display as ipythondisplay


In [38]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0717 13:12:24.566370 140717749794688 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1017'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1017'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

## Rendering OpenAI gym on Colab

Source : https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## Agents

In [0]:
class CartPoleAgent() :
  def __init__(self, obsDims=4, actionDims=2) :
  
    self.discountFactor = 0.95
    self.batchSize = 256
    self.experienceBank = []
    
    xInput = Input(shape=(obsDims,))
    x = xInput
    x = Dense(32, activation='relu')(x)
    x = Dense(actionDims)(x)
  
    self.model = Model(inputs=xInput, outputs=x)
    self.model.summary()
    
    self.model.compile(
        keras.optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.1), 
        loss="mse", 
        metrics=["mse"]
    )

    print("New agent !")
    
    
  def selectAction(self, state, verbose=False) :

    preds = self.model.predict(np.asarray([state]))
    if verbose : print(preds)
    
    action = np.argmax(preds)
    if verbose : print(action)

    return action

  
  def learn(self, prev_state, prev_action, reward, state, action, verbose=False) :
    # This fct implements SARSA    

    # print(f"prev_state : {prev_state} ; prev_action : {prev_action} ; reward : {reward} ; state : {state} : action : {action}")
    
    self.experienceBank.append([prev_state, prev_action, reward, state, action])
    
    prev_states, prev_actions, rewards, states, actions = zip(*sample(self.experienceBank, min(self.batchSize, len(self.experienceBank))))
    
    prev_states = list(prev_states)
    states = list(states)
  
    #val = self.model.predict(np.resize(np.asarray(state), [1, 4]))
    _ = np.asarray(states)
    #print(_)
    #print(_.shape)
    preds = self.model.predict(np.asarray(states))
    #val = val[action]
    #print("actions : ", actions)
    #print("preds : ", preds)
    val = [pred[action] for (pred, action) in zip(preds, actions)] 
    #print(actions)
    
    #target = reward + self.discountFactor * val
    #print("val : ", val)
    targets = [reward + _ * self.discountFactor for reward, _ in zip(rewards, val)]
    #print("targets : ", targets)
    #prev_pred = self.model.predict(np.resize(np.asarray(prev_state), [1, 4]))
    prev_preds = self.model.predict(np.asarray(prev_states))
    if verbose : print("prev_pred ", prev_pred)
    
    #y_true = prev_pred[0]
    y_true = prev_preds
    if verbose : print("y_true ", y_true)
    # y_true[prev_action] = target
    for numb, (y_true_elt, act) in enumerate(zip(y_true, prev_actions)) :
      y_true_elt[act] = targets[numb]
    #y_true = [prev_pred[prev_action] for (prev_pred, prev_action) in zip(prev_preds, prev_actions)] 

    
    #x_train = np.asarray([prev_state])
    x_train = np.asarray(prev_states)
    if verbose :
      print("x ", x_train)
      print("x.shape ", x_train.shape)
    #y_train = np.asarray([y_true])
    y_train = np.asarray(y_true)
    if verbose :
      print("y ", y_train)
      print("y.shape ", y_train.shape)

    tmp = self.model.train_on_batch(x_train, y_train)
    print("tmp : ", tmp)
    
    if verbose : print("Learning occured !!")
    
    


In [49]:
cartPoleAgent = CartPoleAgent()
cartPoleAgent.selectAction([0, 1, 2, 3])
cartPoleAgent.learn([0, 1, 2, 3], 0, 1, [1, 2, 3, 4], 0)
cartPoleAgent.learn([0, 1, 2, 3], 0, 1, [1, 2, 3, 4], 0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 66        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________
New agent !
tmp :  [0.6470175, 0.6470175]
tmp :  [0.6471181, 0.6471181]


## CartPole Simulation

In [55]:
env = wrap_env(gym.make("CartPole-v1"))
# env = gym.make("CartPole-v1")
print(env.action_space)
obs = env.reset()


Discrete(2)


Exception ignored in: <bound method Monitor.__del__ of <Monitor<TimeLimit<CartPoleEnv<CartPole-v1>>>>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 234, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 145, in close
    self._close_video_recorder()
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 217, in _close_video_recorder
    self.video_recorder.close()
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitoring/video_recorder.py", line 129, in close
    os.remove(self.path)
FileNotFoundError: [Errno 2] No such file or directory: '/content/video/openaigym.video.16.134.video000000.mp4'
Exception ignored in: <bound method Monitor.__del__ of <Monitor<TimeLimit<CartPoleEnv<CartPole-v1>>>>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 234, in __del__
    sel

AttributeError: ignored

In [45]:

cartPoleAgent = CartPoleAgent()


env = wrap_env(gym.make("CartPole-v1"))
# env = gym.make(env_name)
obs = env.reset()

#check out the pacman action space!
print(env.action_space)



for episodeNumber in range(3) :
  print(f"Episode {episodeNumber}")
    # SARSA
  prev_obs = None
  prev_action = None
  reward = None
  obs = None
  action = None

  obs = env.reset()
  
  for stepNumber in range(20) :

    prev_obs = obs
    prev_action = action

    action = cartPoleAgent.selectAction(obs)
    # print("action : ", action)
    # print(action)
    obs, reward, done, info = env.step(action) 

    env.render()
    
    cartPoleAgent.learn(prev_obs, prev_action, reward, obs, action) # SARSA

    # print(obs)
    # print(action)

    if done: 
      break;
            
env.close()
show_video()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
dense_27 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 66        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________
New agent !


AttributeError: ignored